In [2]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

import requests
import time
import random
import re
import json
import pandas as pd

from tqdm import tqdm

# 之前使用非官方数据，可以获取历史数据，但由于数据不全最终舍弃了该方法
# url = "http://bz.feigua.cn/ranking/DailyHotVideo/"
# date_id = 20220118
# channel_list = ["155","160","1","3","129","4","36","188","202","119","5","181","167"]

# 需要注意的是在requests当中加入proxy的方法与urllib3有所不同
url = "https://www.bilibili.com/v/popular/rank/"
channel_id = ["guochuang","douga","music","dance","game","knowledge","tech","sports","car","life","food","animal","kichiku","fashion","ent"]
User_agent=[
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]
proxy = [
    {
    'HTTP':'202.108.22.5:80',
    'HTTPS':'202.108.22.5:80'
    },
    {
    'HTTP':'60.255.151.82:80',
    'HTTPS':'60.255.151.82:80'   
    },
    {
    'HTTP':'60.255.151.81:80',
    'HTTPS':'60.255.151.81:80'   
    },
    {
    'HTTP':'58.240.52.114:80',
    'HTTPS':'58.240.52.114:80'   
    },
    {
    'HTTP':'202.108.22.5:80',
    'HTTPS':'202.108.22.5:80'   
    },
]


def request_bilibili_rank(channel_id):
    '''
    :param channel_id: id of channel
    :return: html of website
    '''
    headers = {"User-Agent": random.choice(User_agent)}
    req = requests.get(url+channel_id, headers=headers, proxies=random.choice(proxy))
    html = req.text
    return html

#仿照Youtube视频信息数据集确立bilibili数据集的基本信息项
data_dict = { 
    "video_bv":[],
    "trending_data":[],
    "title":[],
    "channel_title":[],
    "publish_time":[],
    "authors":[],
    "views":[],
    "danmaku":[],
    "likes":[],
    "coins":[],
    "shares":[],
    "tags":[],
    "comment_count":[],
    "description":[],
    "cover_links":[],
    'tid':[],
    'duration':[],
    'favorite':[],
    'scores':[],
    "pop":[]
}

video_bv = [] #视频的bv号，在数据库中可以作为key值
tid = [] #分区id
trending_data = [] #进入排行榜的时间，由于官方的排行榜没有给出历史数据，所以这个时间就是对网页进行爬取的时间
title = [] #视频标题
channel_title = [] #分区名称
publish_time = [] #视频的发布时间
authors = [] #视频作者
views = [] #浏览量
danmaku = [] #弹幕数量
likes = [] #点赞数量
coins = [] #硬币数量
shares = [] #分享数量
tags = [] #视频标签
comment_count = [] #评论数量
description = [] #视频描述
cover_links = [] #视频封面链接
favorite = [] #收藏数量
duration = [] #视频时长
scores = [] #视频排行榜得分
pop = [] #是否进入排行榜

for j in range(len(channel_id)):
    html = request_bilibili_rank(channel_id[j]).replace(" ","")
    #每个视频信息在排行榜中是以json形式的文本展示的
    rankList = r'"rankList":(.*?),"rankNote":'
    pattern = re.compile(rankList,re.S)
    rankList = pattern.findall(html)
    rankList = rankList[0]
    #在python中false要改写为False，null要改写成None
    rankList = rankList.replace('false','False')
    rankList = rankList.replace('null','None')
    rankList = eval(rankList)
    print((j+1)/len(channel_id)*100,"%")
    print('-'*30)

    for i in tqdm(rankList):
        rank_dict = i
        video_bv.append(rank_dict['bvid'])
        tid.append(rank_dict['tid'])
#     stadardTime = time.strftime("%Y-%m-%d %H:%M:%S", rank_dict['ctime'])
        publish_time.append(rank_dict['ctime'])
        title.append(rank_dict['title'])
        channel_title.append(rank_dict['tname'])
        trending_data.append('2022-02-04')
        authors.append(rank_dict['owner']['name'])
        views.append(rank_dict['stat']['view'])
        danmaku.append(rank_dict['stat']['danmaku'])
        favorite.append(rank_dict['stat']['favorite'])
        likes.append(rank_dict['stat']['like'])
        coins.append(rank_dict['stat']['coin'])
        scores.append(rank_dict['score'])
        shares.append(rank_dict['stat']['share'])
        comment_count.append(rank_dict['stat']['reply'])
        cover_links.append(rank_dict['pic'])
        duration.append(rank_dict['duration'])
        pop.append(1)
    

100%|██████████| 100/100 [00:00<00:00, 228821.82it/s]

6.666666666666667 %
------------------------------



100%|██████████| 100/100 [00:00<00:00, 132270.70it/s]

13.333333333333334 %
------------------------------



100%|██████████| 100/100 [00:00<00:00, 196823.28it/s]

...

86.66666666666667 %
------------------------------



100%|██████████| 100/100 [00:00<00:00, 255750.24it/s]

93.33333333333333 %
------------------------------



100%|██████████| 100/100 [00:00<00:00, 238177.40it/s]

100.0 %
------------------------------


In [6]:
# 使用try-except语句确保爬虫不会被中断
Error_bv = []
for i in tqdm(video_bv):
    headers = {"User-Agent": random.choice(User_agent)}
    try:
        html_i = requests.get('https://www.bilibili.com/video/'+i, headers=headers, proxies=random.choice(proxy)).text.replace(" ","")
        try:
            keywords_text = r'name="keywords"content="(.*?)"><'
            pattern = re.compile(keywords_text,re.S)
            keywords_text = pattern.findall(html_i)[0]
            tags.append(keywords_text)
        except:
            tags.append("")
        try:
            desc_text = r'description"content="(.*?)"><'
            pattern = re.compile(desc_text,re.S)
            desc_text = pattern.findall(html_i)[0]
            description.append(desc_text)
        except:
            description.append("")
    except:
        Error_bv.append(i)
        description.append("")
        tags.append("")
        pass
    time.sleep(random.random()+0.5)


 20%|██        | 303/1500 [07:39<27:37,  1.39s/it]


 40%|████      | 605/1500 [16:01<18:53,  1.27s/it]


 60%|██████    | 907/1500 [23:50<14:34,  1.48s/it]


 80%|████████  | 1205/1500 [31:43<07:45,  1.58s/it]


100%|██████████| 1500/1500 [38:33<00:00,  1.62s/it]


In [7]:
data_dict['video_bv'] = video_bv
data_dict['tid'] = tid
data_dict['trending_data'] = trending_data
data_dict['title'] = title
data_dict['channel_title'] = channel_title
data_dict['publish_time'] = publish_time
data_dict['authors'] = authors
data_dict['views'] = views
data_dict['danmaku'] = danmaku
data_dict['likes'] = likes
data_dict['coins'] = coins
data_dict['shares'] = shares
data_dict['tags'] = tags
data_dict['comment_count'] = comment_count
data_dict['description'] = description
data_dict['cover_links'] = cover_links
data_dict['favorite'] = favorite
data_dict['duration'] = duration
data_dict['scores'] = scores
data_dict['pop'] = pop

In [8]:
import pandas as pd
data = pd.DataFrame(data_dict)
data.to_csv('/users/violet/desktop/Graduation_project/pop_sample.csv',index=0,encoding='GB18030')

In [5]:
data_json = json.dumps(data_dict)
file_json = open('/users/violet/desktop/Graduation_project/normal_sample.json','w')
file_json.write(data_json)
file_json.close()

In [3]:
data_dict = json.load(open('/users/violet/desktop/Graduation_project/pop_sample.json'))
call_back = 'jsonCallback_bili_'
e = '%.16f'% random.random()
call_back = call_back + e.split('.')[-1]

channel_id = list(set(data_dict['tid']))

#仿照Youtube视频信息数据集确立bilibili数据集的基本信息项
data_dict = {}

video_bv = [] #视频的bv号，在数据库中可以作为key值
tid = [] #分区id
# get_data = [] #进入排行榜的时间，由于官方的排行榜没有给出历史数据，所以这个时间就是对网页进行爬取的时间
title = [] #视频标题
channel_title = [] #分区名称
publish_time = [] #视频的发布时间
authors = [] #视频作者
views = [] #浏览量
danmaku = [] #弹幕数量
likes = [] #点赞数量
coins = [] #硬币数量
shares = [] #分享数量
tags = [] #视频标签
comment_count = [] #评论数量
description = [] #视频描述
cover_links = [] #视频封面链接
favorite = [] #收藏数量
duration = [] #视频时长
# scores = [] #视频排行榜得分
pop = [] #是否进入排行榜

for k in range(3):
    for j in range(len(channel_id)):
        url = 'https://api.bilibili.com/x/web-interface/dynamic/region?jsonp=jsonp&ps=50&rid='+str(channel_id[j])+'&callback='+call_back
        headers = {
            'authority': 'api.bilibili.com',
    'method': 'GET',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'identity',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'referer': 'https://www.bilibili.com/v/dance',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': random.choice(User_agent),
    'path': '/x/web-interface/dynamic/region?jsonp=jsonp&ps=10&rid=156&callback='+call_back,
    'cookie': '_uuid=FED5E2EC-2D7F-4168-086D-D73100BAC91179797infoc; blackside_state=1; rpdid=|(u)l~lm)Juu0J\'uYJll|J|J~; fingerprint=559d0915c35755a3a9cf38e6d03f6d04; buvid_fp_plain=4976A022-384F-DF2C-DECB-9A6659C027A178523infoc; SESSDATA=63578488%2C1653050587%2C77b40%2Ab1; bili_jct=10cab34d57c07ecc0b38d23cd8f2b62c; DedeUserID=12235886; DedeUserID__ckMd5=c1f2c77973e94aba; sid=kdoqvd8e; video_page_version=v_old_home; LIVE_BUVID=AUTO5716380149776817; CURRENT_QUALITY=80; i-wanna-go-back=-1; b_ut=5; CURRENT_BLACKGAP=0; buvid_fp=98d4a74bfd9219fd32841aa5755a640c; buvid4=F539044A-16CF-84E2-5610-57B3B7903BB774864-022012510-/5r2KmR+xAsnZ7B8RDEMpw%3D%3D; bp_t_offset_12235886=621369677682856211; bsource=share_source_qqchat; buvid3=D57A5DE1-4144-4F8D-BF9B-CBC4A2340613167626infoc; CURRENT_FNVAL=4048; bp_video_offset_12235886=621996661301301900; PVID=1; b_lsid=B10E61012C_17EB30B3984; innersign=0'    
    }

        resp = requests.get(url, headers=headers, proxies=random.choice(proxy))
        jsonStr = re.sub('jsonCallback_bili_.*?\(','',resp.text)
        jsonStr = jsonStr.replace('false','False')
        jsonStr = jsonStr.replace('true','True')
        jsonStr = jsonStr.replace('null','None')
        jsonData = eval(jsonStr[:-1])
        rankList = jsonData['data']['archives']

        for i in tqdm(rankList):
            rank_dict = i
            data_dict[rank_dict['bvid']] = {}
            data_dict[rank_dict['bvid']]['tid'] = (rank_dict['tid'])
    #     stadardTime = time.strftime("%Y-%m-%d %H:%M:%S", rank_dict['ctime'])
            data_dict[rank_dict['bvid']]['publish_time'] = (rank_dict['ctime'])
            data_dict[rank_dict['bvid']]['title'] = (rank_dict['title'])
            data_dict[rank_dict['bvid']]['channel_title'] = (rank_dict['tname'])
            data_dict[rank_dict['bvid']]['get_time'] = '2022-02-04'
            data_dict[rank_dict['bvid']]['authors'] = rank_dict['owner']['name']
            data_dict[rank_dict['bvid']]['views'] = rank_dict['stat']['view']
            data_dict[rank_dict['bvid']]['danmaku'] = rank_dict['stat']['danmaku']
            data_dict[rank_dict['bvid']]['favorite'] = rank_dict['stat']['favorite']
            data_dict[rank_dict['bvid']]['likes'] = rank_dict['stat']['like']
            data_dict[rank_dict['bvid']]['coins'] = rank_dict['stat']['coin']
    #         scores.append(rank_dict['score'])
            data_dict[rank_dict['bvid']]['shares'] = rank_dict['stat']['share']
            data_dict[rank_dict['bvid']]['comment_count'] = rank_dict['stat']['reply']
            data_dict[rank_dict['bvid']]['cover_links'] = rank_dict['pic']
            data_dict[rank_dict['bvid']]['duration'] = rank_dict['duration']
            data_dict[rank_dict['bvid']]['pop'] = 0
        time.sleep(random.random()+0.5)
        print((j+1)*(k*len(channel_id))/(3*len(channel_id))*100,"%")
        print('-'*30)

100%|██████████| 50/50 [00:00<00:00, 122282.92it/s]


NameError: name 'channal_id' is not defined

In [48]:
video_bv = data_dict.keys()
# 使用try-except语句确保爬虫不会被中断
# Error_bv = []
for i in tqdm(Error_bv):
    headers = {"User-Agent": random.choice(User_agent)}
    try:
        html_i = requests.get('https://www.bilibili.com/video/'+i, timeout = 5, headers=headers, proxies=random.choice(proxy)).text.replace(" ","")
        try:
            keywords_text = r'name="keywords"content="(.*?)"><'
            pattern = re.compile(keywords_text,re.S)
            keywords_text = pattern.findall(html_i)[0]
            data_dict[i]['tags']= keywords_text
        except:
            data_dict[i]['tags']= ""
        try:
            desc_text = r'description"content="(.*?)"><'
            pattern = re.compile(desc_text,re.S)
            desc_text = pattern.findall(html_i)[0]
            data_dict[i]['description'] = desc_text
        except:
            data_dict[i]['description']= ""
    except:
        Error_bv.append(i)
    time.sleep(random.random())

100%|██████████| 25/25 [00:25<00:00,  1.07s/it]


In [122]:
data_dict = json.load(open('/users/violet/desktop/Graduation_project/normal_sample.json'))
video_bv = list(data_dict.keys())#视频的bv号，在数据库中可以作为key值
tid = [] #分区id
get_time = [] #进入排行榜的时间，由于官方的排行榜没有给出历史数据，所以这个时间就是对网页进行爬取的时间
title = [] #视频标题
channel_title = [] #分区名称
publish_time = [] #视频的发布时间
authors = [] #视频作者
views = [] #浏览量
danmaku = [] #弹幕数量
likes = [] #点赞数量
coins = [] #硬币数量
shares = [] #分享数量
tags = [] #视频标签
comment_count = [] #评论数量
description = [] #视频描述
cover_links = [] #视频封面链接
favorite = [] #收藏数量
duration = [] #视频时长
# scores = [] #视频排行榜得分
pop = [] #是否进入排行榜
for i in tqdm(video_bv):
#     tags.append(data_dict[i]['tags'])
    tid.append(data_dict[i]['tid'])
    publish_time.append(data_dict[i]['publish_time'])
    title.append(data_dict[i]['title'])
    channel_title.append(data_dict[i]['channel_title'])
    get_time.append(data_dict[i]['get_time'])
    authors.append(data_dict[i]['authors'])
    views.append(data_dict[i]['views'])
    danmaku.append(data_dict[i]['danmaku'])
    favorite.append(data_dict[i]['favorite'])
    likes.append(data_dict[i]['likes'])
    coins.append(data_dict[i]['coins'])
    shares.append(data_dict[i]['shares'])
    comment_count.append(data_dict[i]['comment_count'])
    cover_links.append(data_dict[i]['cover_links'])
    duration.append(data_dict[i]['duration'])
    pop.append(data_dict[i]['pop'])
    tags.append(data_dict[i]['tags'])
    description.append(data_dict[i]['description'])
        

100%|██████████| 9786/9786 [00:00<00:00, 209847.08it/s]


In [124]:
data_dict_ = {}
data_dict_['video_bv'] = video_bv
data_dict_['tid'] = tid
data_dict_['get_time'] = get_time
data_dict_['title'] = title
data_dict_['channel_title'] = channel_title
data_dict_['publish_time'] = publish_time
data_dict_['authors'] = authors
data_dict_['views'] = views
data_dict_['danmaku'] = danmaku
data_dict_['likes'] = likes
data_dict_['coins'] = coins
data_dict_['shares'] = shares
data_dict_['tags'] = tags
data_dict_['comment_count'] = comment_count
data_dict_['description'] = description
data_dict_['cover_links'] = cover_links
data_dict_['favorite'] = favorite
data_dict_['duration'] = duration
data_dict_['pop'] = pop

In [125]:
data_json = json.dumps(data_dict_)
file_json = open('/users/violet/desktop/Graduation_project/normal_sample_.json','w')
file_json.write(data_json)
file_json.close()

In [126]:
import pandas as pd
data = pd.DataFrame(data_dict_)
data.to_csv('/users/violet/desktop/Graduation_project/normal_sample.csv',index=0,encoding='GB18030')

In [76]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context


import requests
import time
from tqdm import tqdm
import random
import re
import json
import pandas as pd

User_agent=[
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]
proxy = [
    {
    'HTTP':'202.108.22.5:80',
    'HTTPS':'202.108.22.5:80'
    },
    {
    'HTTP':'60.255.151.82:80',
    'HTTPS':'60.255.151.82:80'
    },
    {
    'HTTP':'60.255.151.81:80',
    'HTTPS':'60.255.151.81:80'
    },
    {
    'HTTP':'58.240.52.114:80',
    'HTTPS':'58.240.52.114:80'
    },
    {
    'HTTP':'202.108.22.5:80',
    'HTTPS':'202.108.22.5:80'
    },
]

# error_bv = []

def imgDownload(video_bv):
    '''
    :param video_bv: 视频bv号
    :return:
    '''
    for i in range(len(error_bv)):
            img_name=video_bv[i]
            headers = {"User-Agent": random.choice(User_agent)}
            try:
                img_data=requests.get(data_dict[img_name]['cover_links'], timeout=5, headers=headers, proxies=random.choice(proxy))
                with open('/Users/violet/Desktop/Graduation_project/normal_pics/'+img_name+'.jpg', "wb") as file:
                    file.write(img_data.content)
                    file.close()
                    print(img_name + "\tdownload successed!")
                    print((i + 1) / len(error_bv) * 100, "%")
            except Exception as e:
                error_bv.append(img_name)
                print(img_name + "\tdownload failed!", e.args)
                print((i + 1) / len(error_bv) * 100, "%")
            time.sleep(random.random())
data_dict = json.load(open('/users/violet/desktop/Graduation_project/normal_sample.json'))
video_bv = list(error_bv)
print(len(set(video_bv)))
imgDownload(video_bv)


2
BV1ML411u76o	download successed!
50.0 %
BV1oF411v7hi	download successed!
100.0 %


In [129]:
data_dict = json.load(open('/users/violet/desktop/Graduation_project/pop_sample.json'))
data_dict_ = json.load(open('/users/violet/desktop/Graduation_project/normal_sample_.json'))
video_bv = data_dict['video_bv']
video_bv_ = data_dict_['video_bv']
del_list = list(set(video_bv)&set(video_bv_))

In [105]:
import os
import shutil

# 这个函数是用来递归遍历文件夹里面所有内容，包括文件夹和文件，将它们分开放到两个列表
# 顾名思义：file_list 是放文件的、dir_list 是放目录的
def get_file_path(root_path, file_list, dir_list):
    # 获取该目录下所有的文件名称和目录名称
    dir_or_files = os.listdir(root_path);
    for dir_file in dir_or_files:
        # 获取目录或者文件的路径
        dir_file_path = os.path.join(root_path, dir_file)
        # 判断该路径为文件还是路径
        if os.path.isdir(dir_file_path):
            dir_list.append(dir_file_path)
            # 递归获取所有文件和目录的路径
            get_file_path(dir_file_path, file_list, dir_list)
        else:
            file_list.append(dir_file_path);

# 这个函数是用来将遍历好的文件列表中里面某些需要删除的文件进行删除操作
# 如果你不放心带后缀的文件删除后有啥影响，在下面有一句注释的代码，是用来将源文件复制拷贝到某个目录里
def delete_file(file_list):
        # file_list 已经在上面的函数遍历完成 现在里面的内容是所有的文件
        # 我们挨个遍历 找到符合删除文件的条件
    for file_name in file_list:
                # 如果文件的后缀符合要求 就进行操作
        if file_name.endswith(".jpg"):
                        # 在屏幕上打印出来文件的名称 你得知道你删除了什么文件
            print(file_name)
            # 下面代码是将删除这个后缀的文件拷贝到一个备份的文件夹，其实就是多此一举
                        # 一开始是为了确认下删除了那个带后缀的文件会不会对源文件产生一些影响
            try:
                shutil.copy(file_name, r"/users/violet/desktop/Graduation_project/backup_pics/")
                os.remove(file_name)
            except:
                pass

if __name__ == "__main__":
    # 根目录路径
    root_path = r"/users/violet/desktop/Graduation_project/normal_pics/";
    # 用来存放所有的文件路径
    file_list = []
    for i in tqdm(del_list):
        file_list.append(root_path+i+'.jpg')
    # 用来存放所有的目录路径
#     dir_list = []
#     get_file_path(root_path, file_list, dir_list)
    # print(file_list)
    # print(dir_list)
    delete_file(file_list)
    os.system("pause")

100%|██████████| 262/262 [00:00<00:00, 112213.59it/s]


/users/violet/desktop/Graduation_project/normal_pics/BV1b3411E7d5.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1bY411t7Dr.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV14P4y177xW.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1cP4y1J7dQ.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1qb4y1J7kn.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1244y1p7kt.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1mq4y1F73x.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1NR4y1T7M9.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1VP4y1A7CK.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1y44y1p7k4.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1xa41127aL.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1Za411177Z.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1qL411c7CS.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV16T4y1k7vS.jpg
/users/violet/deskto

/users/violet/desktop/Graduation_project/normal_pics/BV1TF411p73K.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1i34y117MV.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1zP4y1N7Wf.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1B44y157G5.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1eT4y1k7Ms.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1UZ4y1d7CU.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV12L4y1W7jG.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1jT4y117Dk.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1wT4y1C7Us.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV14r4y1Y7uL.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1WR4y1j7Ar.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1xb4y1n7CL.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1f44y1W7jt.jpg
/users/violet/desktop/Graduation_project/normal_pics/BV1jq4y1c77T.jpg
/users/violet/deskto